In [51]:
#Marnt Fendl Code

#Imports der benötigten Bibliotheken
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [52]:
#Pfade der Inputs festlegen
partnerlist = r'Partnerlisten\Top2000AS.xlsx'
keywordslist = 'Keywords/CradlepointKeywords.xlsx'

In [58]:
#import Excel Partnerliste to partner_list array

df = pd.read_excel (partnerlist, header = None)
partner_list = df[0].tolist()
partner_list = [each_string.lower() for each_string in partner_list]

# initializing special characters
special_char = '@_!#$%^&*()<>?/\|}{~:;.[]'
 
# remove special characters 
partner_list = [''.join(filter(lambda i: i not in special_char, string)) for string in partner_list]

print(len(partner_list))
print(partner_list)

2000
['sva system vertrieb alexander gmbh', 'cancom gmbh', 'bechtle logistik  service gmbh', 'timetoact software ', 'profi engineering systems ag', 'computacenter ag  coohg', 'logicalis gmbh', 'softwareone deutschland gmbh', 'deutsche telekom business solutions', 'acp holding deutschland gmbh', 'pro-com datensysteme gmbh', 'pdv-systeme gmbh', 'netgo gmbh', 'noris network ag', 'axians cloud  it-automation', 'hg hansen  gieraths', 'comline computer +', 'esell gmbh', 'sandata edv-systemhaus gmbh', 'concat ag', 'softwareone deutschland gmbh', 'dell products', 'dualutions gmbh', 'systema', 'dextradata gmbh', 'mentiq gmbh', 'netgo gmbh', 'kramer  crew gmbh  co kg', 'koala management solutions gmbh', 'dts systeme gmbh', 'blue consult gmbh', '8keys gmbh', 'patrick straub', 'ars computer und', 'advanced unibyte gmbh', 'bizteam systemhaus gmbh', 'shd system-haus-dresden gmbh', 'atos information technology gmbh', 'retarus gmbh', 'scaltel ag', 'it-haus gmbh', 'aura computersysteme gmbh', 'trigonov

In [54]:
#Bearbeitung der Sourcecodesnamen

#Array mit Sourcecodesnamen erstellen
sourcecodes_list = os.listdir('Sourcecodes')
#.txt Endung von Namen entfernen
sourcecodes_list = {x.replace('.txt', '') for x in sourcecodes_list}
#Ausgabe der Liste und Länge der Liste
print(len(sourcecodes_list))
print(sourcecodes_list)

2171
{'navum gmbh', 'xplus1 gmbh', 'bluechip computer ag', 'codecentric ag', 'kutzschbach', 'blue consult gmbh', 'computer wunsch', 'tiefenbach it gmbh', 'cube office gmbh  cokg', 'cth riesa gmbh', 'faulhaber gmbh', 'capricorn handels gmbh', 'protechnology gmbh', 'binsec gmbh', 'lead on gmbh', 'rockitconsulting gmbh', 'think 1st ltd  cokg', 'lt memory gmbh hard- und', 'dyna bcs informatik gmbh', 'bi plus gmbh', 'viridicon ag', 'nxtgn solutions gmbh', 'elkomsolutions gmbh', 'jk-computer gmbh', 'tradeo gmbh', 'releon gmbh  co kg', 'hottgenroth software gmbh  co kg', 'mitc gmbh', 'dr haasters  partner gmbh', 'thomas stollorcz', 'amexus informationstechnik', 'tieto germany gmbh', 'datagroup hamburg gmbh', 'herrmann  lenz services', 'blue it services gmbh', 'pik ag', 'bytesfactory ek', 'analytek gmbh', 'porthun  thiede', 'meier it-service', 'udo gärtner gmbh', 'consulting 86 gmbh', 'coosec one gmbh', 'noris network ag', 'pds novis it-systeme gmbh', 'chiemgau edv gmbh  co kg', 'skill compute

In [55]:
#Check wieviele der gewollte Sourcecodes sind schon runtergeladen
counter = 0
for partner in partner_list:
    if partner in sourcecodes_list:
        counter +=1

print(f'{counter} {len(partner_list)}')

1909 2000


In [59]:
#Runterladen der Sourcecodes, welche noch nicht gefunden wurden 

#Festlegen von Arrays für die spätere Überprüfung der Partner
found = []
not_found = []

#Loop durch alle Partner in der Partnerliste
for index,company in enumerate(partner_list):
    #Wenn Sourcecode noch nicht runtergeladen, wird der Sroucecode runtergeladen
    if company not in sourcecodes_list:
        #Companyname Space mit Plus ersetzten, damit googlen möglich
        company.replace(' ', '+')
        #Leider hat die es mit Initalisierung außerhalb nicht geklappt
        driver = webdriver.Chrome()
        google = 'https://www.google.com/search?q='
        url = google+company
        #Aufrufen der Googlesuche
        driver.get(url)
        print(url)
        #Array mit allen gefundenen Links der Googlesuche erstellen
        results = driver.find_elements_by_partial_link_text('http')
        if results:
            print(results[0])
            #ersten Link aufrufen
            richtiger_link = results[0].get_attribute('href')
            print(richtiger_link)
            try:
                driver.get(richtiger_link)
                driver.implicitly_wait(2)
                company = company.replace('+', '')
                filename = f'{sourcecodes}\\{company}.txt'
                with open(filename,'w', encoding='utf-8') as f:
                    datafile = f.write(driver.page_source.lower())
                    f.close()
                driver.close()
                print(f'{index+1} von {len(partner_list)} fertig')
                found.append(company)
            except:
                print('Konnte Scourecodes nicht downloaden')
                not_found.append(company)
        else:
            print(f'Es wurde kein Link für {company} gefunden')
            not_found.append(company)
            
    else:
        next
        print(index)
        found.append(company)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
https://www.google.com/search?q=comline computer +
<selenium.webdriver.remote.webelement.WebElement (session="f729910c543c88b19c6ee962c006640e", element="576e684b-44d0-4f73-b202-4aded9dff55f")>
https://www.comline-se.de/
17 von 2000 fertig
17
18
19
20
21
22
23
24
25
26
27
28
29
30
https://www.google.com/search?q=8keys gmbh
<selenium.webdriver.remote.webelement.WebElement (session="c0edbac14699688d73e9aaff45559453", element="4e74c1fd-c9fe-4f2b-8b46-17fd41212ce8")>
https://www.8keys.de/
32 von 2000 fertig
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
https://www.google.com/search?q=ntt germany ag  cokg
Es wurde kein Link für ntt g

808 von 2000 fertig
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
https://www.google.com/search?q=claus magin
Es wurde kein Link für claus magin gefunden
856
857
858
859
860
861
862
863
https://www.google.com/search?q=prologistik gmbh + cokg
<selenium.webdriver.remote.webelement.WebElement (session="b7f5cf56740a7211b3406d18e67d33f9", element="32d7e6a9-3c9b-404f-ad9a-7f150fe1d56d")>
https://www.prologistik.com/
865 von 2000 fertig
865
https://www.google.com/search?q=u + p systemhaus gmbh
<selenium.webdriver.remote.webelement.WebElement (session="3a9cbb9326d152f1611e1cf1220424ec", element="4f116147-632d-4d8f-9faa-586b45c8150c")>
https://www.juunit.com/loesungen/systemhauswechsel
867 von 2000 fertig
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
https://www.google.com/search?q=digital advice gmbh  cokg
Es wurde kein Link für dig

https://www.google.com/search?q=frank hofmann
<selenium.webdriver.remote.webelement.WebElement (session="96511b6ecc9d349122b0fd1f40292664", element="7bd66a25-c10e-4101-9ca0-49e32951b947")>
https://de.wikipedia.org/wiki/Frank_Hofmann_(Journalist)
1407 von 2000 fertig
1407
1408
1409
1410
1411
1412
1413
1414
1415
1416
1417
https://www.google.com/search?q=unite+ it services gmbh
<selenium.webdriver.remote.webelement.WebElement (session="fe0e2445cb40c70d6f9e4e625d31757e", element="7d515b18-a833-40e1-87a7-6e6c38d67397")>
https://www.juunit.com/
1419 von 2000 fertig
1419
1420
1421
1422
1423
1424
1425
1426
1427
1428
1429
1430
1431
1432
1433
https://www.google.com/search?q=dns datentechnik, systemhaus und
Es wurde kein Link für dns datentechnik, systemhaus und gefunden
1435
1436
1437
1438
1439
1440
1441
1442
1443
1444
1445
1446
1447
1448
1449
1450
https://www.google.com/search?q=bitfabrik gmbh  cokg
Es wurde kein Link für bitfabrik gmbh  cokg gefunden
https://www.google.com/search?q=jens weber


https://www.google.com/search?q=4service-systems gmbh
<selenium.webdriver.remote.webelement.WebElement (session="d163169424a4235a769105a1c9cda8a5", element="e77f28db-3ef2-4fa0-add0-a70a52d02083")>
http://www.4service-systems.de/
1952 von 2000 fertig
1952
1953
1954
https://www.google.com/search?q=ruwa unternehmensberatung für
Es wurde kein Link für ruwa unternehmensberatung für gefunden
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
https://www.google.com/search?q=klw gmbh
Es wurde kein Link für klw gmbh gefunden
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999


In [61]:
#Wieviele Sourcecodes wurden gedownloaded?
print(f'Zu {len(found)} Partnern wurden Sourcecodes gedownloaded')
print(f'Zu {len(not_found)} Partnern wurden keine Sourcecodes gedownloaded')

Zu 1959 Partnern wurden Sourcecodes gedownloaded
Zu 41 Partnern wurden keine Sourcecodes gedownloaded


In [62]:
#Erstellen einer Funktion, welche den Sourcecode nach Keywords durchsucht und aus Findings ein Dictionary mit Keywords
#und Partnernamen erstellt
#hier kann man auch noch einen Counter einbauen, wie oft das Wort gefunden wurde
def check (filename, keywords):
    my_dict = {}
    with open(filename, encoding='utf-8') as f:
        datafile = f.readlines()
    for word in keywords:
        my_dict[word] = False
        for line in datafile:
            if word in line:
                my_dict[word] = True
    return my_dict

In [65]:
#Excel Keywords in Array 'Keywords' importieren
df = pd.read_excel (keywordslist, header = None)
keywords = []
keywords = df[0].tolist()
keywords = [str(i) for i in keywords]

print(keywords)

['cradlepoint', '5g', 'wireless wan', 'sd-wan', 'sd wan', 'wan', 'wired wan', 'wan service', '4g', 'lte', '5g router', 'router', 'High-Band 5G', '99999', '99.99', 'downtime', 'meraki', 'cisco', 'fortinet', 'digi', 'cellular networking', 'cellular network', 'networking', 'netzwerk', 'vernetzung', 'networks', 'network']


In [66]:
#Erneut Sourcecodes in Array importieren für spätere Erstellung des gesamten Dictionarys
#Damit man den Code von oben nach unten ausführen kann
sourcecodes_list = os.listdir('Sourcecodes')
print(len(sourcecodes_list))
print(sourcecodes_list)

2197
['10m gmbh - document management.txt', '20-20 technologies gmbh.txt', '21torr gmbh.txt', '22nd human network gmbh .txt', '250TopPartner', '2f-it gmbh.txt', '3i informationstechnologie gmbh.txt', '3net gmbh.txt', '3u telecom gmbh.txt', '4brands reply.txt', '4c group ag.txt', '4sellers gmbh.txt', '4service-systems gmbh.txt', '5g it service gmbh.txt', '8com gmbh und co kg.txt', '8keys gmbh.txt', '8soft gmbh.txt', 'aaic soft systems gmbh.txt', 'aareon deutschland gmbh.txt', 'abacon ges f mess-.txt', 'abakus it ag.txt', 'abamedicus heiko menge .txt', 'abas systemhaus gmbh.txt', 'abass gmbh .txt', 'abass gmbh.txt', 'abidat gmbh.txt', 'abilis gmbh.txt', 'abisys gmbh.txt', 'abl social federation gmbh.txt', 'abona deutschland gmbh.txt', 'abtis gmbh.txt', 'accelerate it consulting.txt', 'accenture gmbh.txt', 'acceptit gmbh.txt', 'accxess it-systems gmbh.txt', 'acotel computer vertriebs gmbh.txt', 'acp holding deutschland gmbh.txt', 'acp it solutions ag.txt', 'acp it solutions gmbh ao.txt', 

In [68]:
#Die Check Funktion auf alle Partner der Partnerliste ausführen und aus den einzelen Dictionarys ein großes Dictionary für jeden
#Partner erstellen

complete_dict = {}
not_analyzed = []

for current_company in partner_list:
    try:
        #current_company = current_company.replace(' ','+')
        company_path = f'Sourcecodes\\{current_company}.txt'
        current_dict = check(company_path, keywords)
        complete_dict[current_company] = current_dict
    except:
        print(f'{current_company} hat nicht funktioniert')
        not_analyzed.append(current_company)

comline computer + hat nicht funktioniert
ntt germany ag  cokg hat nicht funktioniert
lettner edv-technik gmbh hat nicht funktioniert
h+m computersysteme gmbh hat nicht funktioniert
lunzer + partner gmbh hat nicht funktioniert
list + lohr gmbh hat nicht funktioniert
netline computer hat nicht funktioniert
bright skies gmbh hat nicht funktioniert
logistic networking hat nicht funktioniert
b+w it solutions gmbh hat nicht funktioniert
sperl + krieglmeier büro- und hat nicht funktioniert
hosysteme gmbh + co kg hat nicht funktioniert
coplan computer + hat nicht funktioniert
heinrich steuber gmbh + co hat nicht funktioniert
bic4 broadcast + it consulting gmbh hat nicht funktioniert
claus magin hat nicht funktioniert
prologistik gmbh + cokg hat nicht funktioniert
u + p systemhaus gmbh hat nicht funktioniert
digital advice gmbh  cokg hat nicht funktioniert
cms sudhaus  partner gmbh hat nicht funktioniert
s+c softwareentwicklung und hat nicht funktioniert
g + b medientechnik gmbh hat nicht funk

In [43]:
#Dictionary in ein Pandas Dataframe umwandeln, damit man dieses als Excel exportieren kann 
data = pd.DataFrame.from_dict(complete_dict, orient='index')
print(data)
data.to_excel(f'Auswertungen/Top2000AS.xlsx') #Namen ändern für neue Datei

                                    cradlepoint     5g  wireless wan  sd-wan  \
sva system vertrieb alexander gmbh        False   True         False    True   
cancom gmbh                               False   True         False    True   
bechtle logistik  service gmbh            False  False         False    True   
timetoact software                        False   True         False   False   
profi engineering systems ag              False  False         False   False   
...                                         ...    ...           ...     ...   
daphi gmbh                                False  False         False   False   
aspedi gmbh                               False  False         False   False   
businessvalues it-services gmbh           False  False         False   False   
condat datensysteme gmbh                  False  False         False   False   
netrix it-service gmbh                    False  False         False   False   

                                    sd 